VLM

In [1]:
import torch
from torch.nn.utils import clip_grad_value_
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=48,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-28 21:50:02 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
model.config.image_token_index = 16 # new image token
model.language_model.lm_head.weight.requires_grad_(True);
model.language_model.model.embed_tokens.padding_idx = None
# model.language_model.lm_head.load_state_dict(torch.load("/home/zhenlan/Desktop/Projects/ARC2/Model/gemma48.pth"))
model.language_model.lm_head.load_state_dict(torch.load("../../Model/lm_heads_weights_VLM_oneshot.pth")) 

<All keys matched successfully>

In [3]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

#### Fine-tune embedding

In [4]:
epochs = 10
accumulation_steps = 32
lr = 6e-5
clip = 6e-5
MAX_LEN = 3600

In [5]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [6]:
# time tracking
start_time = time.time()
train_loss1 = 0
train_loss2 = 0
val_loss1 = 0
val_loss2 = 0
for epoch in range(epochs):
    model.train()
    for i,(inputs, y, l) in enumerate(data_gen_VLM(data, True, tokenizer, 3, tokenize_func=tokenize_VLM_oneshot)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(**inputs).logits
            _,L,V = yhat.shape
            loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
            loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
            loss = loss1 * 1.0 + loss2
        loss.backward()
        train_loss1 += loss1.item()
        train_loss2 += loss2.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(inputs, y, l) in enumerate(data_gen_VLM(data,False,tokenizer,3, tokenize_func=tokenize_VLM_oneshot)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(**inputs).logits
                _,L,V = yhat.shape
                loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
                loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
        val_loss1 += loss1.item()
        val_loss2 += loss2.item()    
    print(f"Epoch {epoch+1}, Train Loss1: {train_loss1/(i+1):.2f}, Train Loss2: {train_loss2/(i+1):.2f}, Val Loss1: {val_loss1/(j+1):.2f}, Val Loss2: {val_loss2/(j+1):.2f}")
    train_loss1 = 0
    train_loss2 = 0
    val_loss1 = 0
    val_loss2 = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1, Train Loss1: 1.15, Train Loss2: 7.25, Val Loss1: 0.66, Val Loss2: 2.66
Epoch 2, Train Loss1: 1.10, Train Loss2: 2.33, Val Loss1: 0.64, Val Loss2: 2.10
Epoch 3, Train Loss1: 1.08, Train Loss2: 2.03, Val Loss1: 0.63, Val Loss2: 1.98
Epoch 4, Train Loss1: 1.06, Train Loss2: 1.93, Val Loss1: 0.63, Val Loss2: 1.90
Epoch 5, Train Loss1: 1.05, Train Loss2: 1.89, Val Loss1: 0.62, Val Loss2: 1.85
Epoch 6, Train Loss1: 1.03, Train Loss2: 1.84, Val Loss1: 0.62, Val Loss2: 1.88
Epoch 7, Train Loss1: 1.01, Train Loss2: 1.78, Val Loss1: 0.61, Val Loss2: 1.82
Epoch 8, Train Loss1: 1.00, Train Loss2: 1.75, Val Loss1: 0.61, Val Loss2: 1.84
Epoch 9, Train Loss1: 0.98, Train Loss2: 1.75, Val Loss1: 0.61, Val Loss2: 1.82
Epoch 10, Train Loss1: 0.97, Train Loss2: 1.73, Val Loss1: 0.60, Val Loss2: 1.83
Training completed in 159.55 minutes


In [7]:
torch.save(model.language_model.lm_head.state_dict(), '../../Model/lm_heads_weights_VLM_oneshot.pth')

#### Fine-Tune QLORA

In [4]:
epochs = 60
accumulation_steps = 32
lr = 6e-5
clip = 6e-3
MAX_LEN = 3600

In [5]:
model.max_seq_length = MAX_LEN

In [6]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           # Larger = higher accuracy, but might overfit
    lora_alpha = 64,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [7]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [8]:
# time tracking
start_time = time.time()
train_loss1 = 0
train_loss2 = 0
val_loss1 = 0
val_loss2 = 0
for epoch in range(epochs):
    model.train()
    for i,(inputs, y, l) in enumerate(data_gen_VLM(data, True, tokenizer, 3, tokenize_func=tokenize_VLM_oneshot)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(**inputs).logits
            _,L,V = yhat.shape
            loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
            loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
            loss = loss1 * 0.25 + loss2
        loss.backward()
        train_loss1 += loss1.item()
        train_loss2 += loss2.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(inputs, y, l) in enumerate(data_gen_VLM(data,False,tokenizer,3, tokenize_func=tokenize_VLM_oneshot)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(**inputs).logits
                _,L,V = yhat.shape
                loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
                loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
        val_loss1 += loss1.item()
        val_loss2 += loss2.item()    
    print(f"Epoch {epoch+1}, Train Loss1: {train_loss1/(i+1):.2f}, Train Loss2: {train_loss2/(i+1):.2f}, Val Loss1: {val_loss1/(j+1):.2f}, Val Loss2: {val_loss2/(j+1):.2f}")
    train_loss1 = 0
    train_loss2 = 0
    val_loss1 = 0
    val_loss2 = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1, Train Loss1: 0.82, Train Loss2: 1.49, Val Loss1: 0.51, Val Loss2: 1.47
Epoch 2, Train Loss1: 0.68, Train Loss2: 1.33, Val Loss1: 0.47, Val Loss2: 1.38
Epoch 3, Train Loss1: 0.63, Train Loss2: 1.24, Val Loss1: 0.47, Val Loss2: 1.33
Epoch 4, Train Loss1: 0.60, Train Loss2: 1.18, Val Loss1: 0.45, Val Loss2: 1.32
Epoch 5, Train Loss1: 0.58, Train Loss2: 1.17, Val Loss1: 0.44, Val Loss2: 1.30
Epoch 6, Train Loss1: 0.56, Train Loss2: 1.13, Val Loss1: 0.43, Val Loss2: 1.32
Epoch 7, Train Loss1: 0.55, Train Loss2: 1.12, Val Loss1: 0.42, Val Loss2: 1.30
Epoch 8, Train Loss1: 0.54, Train Loss2: 1.10, Val Loss1: 0.41, Val Loss2: 1.28
Epoch 9, Train Loss1: 0.53, Train Loss2: 1.08, Val Loss1: 0.41, Val Loss2: 1.26
Epoch 10, Train Loss1: 0.53, Train Loss2: 1.06, Val Loss1: 0.41, Val Loss2: 1.25
Epoch 11, Train Loss1: 0.52, Train Loss2: 1.05, Val Loss1: 0.41, Val Loss2: 1.25
Epoch 12, Train Loss1: 0.51, Train Loss2: 1.02, Val Loss1: 0.40,

KeyboardInterrupt: 

In [9]:
model.save_pretrained("../../Model/merged_model_VLM_oneshot")
torch.save(model.language_model.lm_head.state_dict(), '../../Model/lm_heads_weights_VLM_oneshot.pth')

#### continue training

In [4]:
epochs = 20
accumulation_steps = 32
lr = 3e-5
clip = 3e-3
MAX_LEN = 3600
model.max_seq_length = MAX_LEN

In [5]:
# model.language_model.lm_head.load_state_dict(torch.load("../../Model/lm_heads_weights_VLM_oneshot.pth")) 
model = PeftModel.from_pretrained(model, "../../Model/merged_model_VLM2", is_trainable=True)

In [6]:
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

In [7]:
# time tracking
start_time = time.time()
train_loss1 = 0
train_loss2 = 0
val_loss1 = 0
val_loss2 = 0
for epoch in range(epochs):
    model.train()
    for i,(inputs, y, l) in enumerate(data_gen_VLM(data, True, tokenizer, 3, tokenize_func=tokenize_VLM_oneshot)):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            yhat = model(**inputs).logits
            _,L,V = yhat.shape
            loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
            loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
            loss = loss1 * 0.1 + loss2
        loss.backward()
        train_loss1 += loss1.item()
        train_loss2 += loss2.item()

        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    for j,(inputs, y, l) in enumerate(data_gen_VLM(data,False,tokenizer,3, tokenize_func=tokenize_VLM_oneshot)):
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                yhat = model(**inputs).logits
                _,L,V = yhat.shape
                loss1 = loss_fn(yhat[0,:l].view(l, V),y[0,:l].view(-1))
                loss2 = loss_fn(yhat[0,l:].view(L-l, V),y[0,l:].view(-1))
        val_loss1 += loss1.item()
        val_loss2 += loss2.item()    
    print(f"Epoch {epoch+1}, Train Loss1: {train_loss1/(i+1):.2f}, Train Loss2: {train_loss2/(i+1):.2f}, Val Loss1: {val_loss1/(j+1):.2f}, Val Loss2: {val_loss2/(j+1):.2f}")
    train_loss1 = 0
    train_loss2 = 0
    val_loss1 = 0
    val_loss2 = 0
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!
Epoch 1, Train Loss1: 0.67, Train Loss2: 1.57, Val Loss1: 0.32, Val Loss2: 1.52


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("../../Model/merged_model_VLM_oneshot")
torch.save(model.language_model.lm_head.state_dict(), '../../Model/lm_heads_weights_VLM_oneshot.pth')